In [0]:
# Access to Google Colab
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
# Test if GPU is active
import tensorflow as tf
tf.test.gpu_device_name()

import keras
print (keras.__version__)

2.1.6


Using TensorFlow backend.


In [0]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io
import zipfile
from skimage import color
from matplotlib import pyplot
import time
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD


%matplotlib inline

In [4]:
# Import data and label files

# Train labels
training_labels_downloaded = drive.CreateFile({'id': '1rJCglyD-6uTow7RFsj5K7WxSl1YFyg8M'})
training_labels_downloaded.GetContentFile('train.txt')
train_labels = pd.read_csv('train.txt', sep=" ", header=None)

# Test labels
test_labels_downloaded = drive.CreateFile({'id': '15a5U1SsguFSdQim7ifoA9vpuhyL5AgTZ'})
test_labels_downloaded.GetContentFile('vali.txt')
test_labels = pd.read_csv('vali.txt', sep=" ", header=None)

# Train data
train_data_downloaded = drive.CreateFile({'id': '1-X94bMCmjMShVVN5VJXPZ936Zly0BOP_'})
train_data_downloaded.GetContentFile('train-set.zip')
train_data = zipfile.ZipFile('train-set.zip', 'r')
train_data.extractall()

# Test data
test_data_downloaded = drive.CreateFile({'id': '1lSMxDwZFD40ceFAlWCoqT1TlJ9gqgqOx'})
test_data_downloaded.GetContentFile('vali-set.zip')
test_data = zipfile.ZipFile('vali-set.zip', 'r')
test_data.extractall()

rm: cannot remove 'train-set/__MACOSX': No such file or directory


In [5]:
!ls -a

.	.config   .ipython  __MACOSX  train-set      vali-set
..	datalab   .keras    .nv       train-set.zip  vali-set.zip
.cache	.forever  .local    .rnd      train.txt      vali.txt


In [6]:
%%time
# Import and convert data images into RGB arrays
# Under LeNet, we read the sample in binary form with only one channel using mode ='L'
import scipy.misc

# Train data
train_images = []
for i in range (0, len(train_labels)):
  im = scipy.misc.imread('train-set/train-set/'+train_labels[0][i], flatten=False, mode='L')
  train_images.append(im.astype('uint8'))
   

# Test data
test_images = []
for i in range (0, len(test_labels)):
  im = scipy.misc.imread('vali-set/vali-set/'+test_labels[0][i], flatten=False, mode='L')
  test_images.append(im.astype('uint8'))

CPU times: user 14.7 s, sys: 1.71 s, total: 16.4 s
Wall time: 16.4 s


In [0]:
# Define train and test sets
y_train = train_labels[1][:]
y_test = test_labels[1][:]
x_train = (np.asarray(train_images)).reshape(len(train_images), len(train_images[0]), len(train_images[0]), 1)
x_test = (np.asarray(test_images)).reshape(len(test_images), len(test_images[0]), len(test_images[0]), 1)


class_num = len(np.unique(y_train))

In [8]:
# One-hot code the labels
y_train = keras.utils.to_categorical(y_train,class_num)
y_test = keras.utils.to_categorical(y_test,class_num)

# Print shapes of data and labels
print('Train data: ',x_train.shape)
print('Train labels: ',y_train.shape)
print('Test data: ',x_test.shape)
print('Test labels: ',y_test.shape)

Train data:  (37882, 128, 128, 1)
Train labels:  (37882, 62)
Test data:  (6262, 128, 128, 1)
Test labels:  (6262, 62)


In [9]:
# Build a modified LeNet architecture with additional Dropout, batch normalization, Relu activiation function
# Details of the full model is decribed in the report
# When conduct experiments, we could simply delete the layer by adding '#' tounscript orline add a layer by using 'modellent5.add()' 
# Results from the experiments were also included in the report


# Use Max Pooling Layers to resize the inputs to 32*32
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD


modellenet5 = Sequential()
# Use Max Pooling Layers to resize the inputs to 32*32
modellenet5.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), input_shape=(128, 128, 1)))
modellenet5.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
modellenet5.add(BatchNormalization())

# Start of Original Lenet5
modellenet5.add(Conv2D(6, (5, 5), activation='relu'))
modellenet5.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
modellenet5.add(Dropout(0.25))
modellenet5.add(BatchNormalization())

modellenet5.add(Conv2D(16, (5, 5), activation='relu'))
modellenet5.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

modellenet5.add(Flatten())
modellenet5.add(Dense(256, activation='relu'))
modellenet5.add(Dropout(0.25))
modellenet5.add(BatchNormalization())

modellenet5.add(Dense(62, activation='softmax'))

# Define 2 optimizing functions: 'adam' & 'sgd'
# Hypermeters within these functions can be changed
# Another optimzer 'rmsprop' we use default hyperparater values

adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)


# Complie the model
modellenet5.compile(loss='categorical_crossentropy', optimizer= 'rmsprop',metrics=['accuracy'])



# Check runtime
start = time.time()

# Train model
modellenet5.fit(x_train,y_train,batch_size=128,epochs=20,
          validation_data=(x_test, y_test),
          verbose=1)
score = modellenet5.evaluate(x_test, y_test, verbose=0)
print('Test loss: {:0.2f}%'.format(score[0]*100))
print('Test accuracy: {:0.2f}%'.format(score[1]*100))

# Print runtime
end = time.time()
print('Total runtime is {:0.2f} minutes'.format(int(end-start)/60))

Train on 37882 samples, validate on 6262 samples
Epoch 1/20
37882/37882 [==============================] - 8s 207us/step - loss: 1.4845 - acc: 0.6166 - val_loss: 0.8220 - val_acc: 0.7637
Epoch 2/20
37882/37882 [==============================] - 6s 159us/step - loss: 0.8323 - acc: 0.7565 - val_loss: 0.6631 - val_acc: 0.7966
Epoch 3/20
37882/37882 [==============================] - 6s 160us/step - loss: 0.7178 - acc: 0.7821 - val_loss: 0.5998 - val_acc: 0.8128
Epoch 4/20
10112/37882 [=======>......................] - ETA: 4s - loss: 0.6527 - acc: 0.8007

37882/37882 [==============================] - 6s 159us/step - loss: 0.6486 - acc: 0.7998 - val_loss: 0.5862 - val_acc: 0.8141
Epoch 5/20
37882/37882 [==============================] - 6s 157us/step - loss: 0.6123 - acc: 0.8077 - val_loss: 0.5405 - val_acc: 0.8279
Epoch 6/20
37882/37882 [==============================] - 6s 157us/step - loss: 0.5704 - acc: 0.8174 - val_loss: 0.5221 - val_acc: 0.8293
Epoch 7/20
27008/37882 [====================>.........] - ETA: 1s - loss: 0.5477 - acc: 0.8237

37882/37882 [==============================] - 6s 159us/step - loss: 0.5528 - acc: 0.8212 - val_loss: 0.5080 - val_acc: 0.8397
Epoch 8/20
37882/37882 [==============================] - 6s 158us/step - loss: 0.5276 - acc: 0.8283 - val_loss: 0.4940 - val_acc: 0.8400
Epoch 9/20
37882/37882 [==============================] - 6s 158us/step - loss: 0.5115 - acc: 0.8307 - val_loss: 0.4798 - val_acc: 0.8406
Epoch 10/20
30080/37882 [======================>.......] - ETA: 1s - loss: 0.4934 - acc: 0.8371

37882/37882 [==============================] - 6s 159us/step - loss: 0.4989 - acc: 0.8357 - val_loss: 0.4721 - val_acc: 0.8478
Epoch 11/20
37882/37882 [==============================] - 6s 158us/step - loss: 0.4808 - acc: 0.8398 - val_loss: 0.4689 - val_acc: 0.8465
Epoch 12/20
37882/37882 [==============================] - 6s 159us/step - loss: 0.4733 - acc: 0.8426 - val_loss: 0.4752 - val_acc: 0.8451
Epoch 13/20
30080/37882 [======================>.......] - ETA: 1s - loss: 0.4557 - acc: 0.8480

37882/37882 [==============================] - 6s 158us/step - loss: 0.4583 - acc: 0.8469 - val_loss: 0.4597 - val_acc: 0.8433
Epoch 14/20
37882/37882 [==============================] - 6s 159us/step - loss: 0.4533 - acc: 0.8474 - val_loss: 0.4581 - val_acc: 0.8508
Epoch 15/20
37882/37882 [==============================] - 6s 159us/step - loss: 0.4413 - acc: 0.8499 - val_loss: 0.4594 - val_acc: 0.8515
Epoch 16/20
30080/37882 [======================>.......] - ETA: 1s - loss: 0.4358 - acc: 0.8517

37882/37882 [==============================] - 6s 157us/step - loss: 0.4367 - acc: 0.8523 - val_loss: 0.4449 - val_acc: 0.8536
Epoch 17/20
37882/37882 [==============================] - 6s 158us/step - loss: 0.4269 - acc: 0.8539 - val_loss: 0.4469 - val_acc: 0.8523
Epoch 18/20
37882/37882 [==============================] - 6s 157us/step - loss: 0.4227 - acc: 0.8564 - val_loss: 0.4588 - val_acc: 0.8499
Epoch 19/20
30080/37882 [======================>.......] - ETA: 1s - loss: 0.4151 - acc: 0.8583

37882/37882 [==============================] - 6s 157us/step - loss: 0.4168 - acc: 0.8570 - val_loss: 0.4437 - val_acc: 0.8534
Epoch 20/20
37882/37882 [==============================] - 6s 159us/step - loss: 0.4069 - acc: 0.8593 - val_loss: 0.4521 - val_acc: 0.8576
Test loss: 45.21%
Test accuracy: 85.76%
Total runtime is 2.05 minutes


In [10]:
# Another way to resize the original images to 32*32 for the LeNet 5 model is to use a resize library


# Use scipy.misc.imresize() approach
from PIL import Image
import os, sys

def imread(path):  
    img = scipy.misc.imread(path).astype(np.float)  
    return img  
  

train_images_resize = []
for i in range (0, len(train_labels)):       
        content_image = imread('train-set/train-set/'+train_labels[0][i],)
        im = scipy.misc.imresize(content_image, (32,32))
        train_images_resize.append(im.astype('uint8'))
        
        
test_images_resize = []
for i in range (0, len(test_labels)):       
        content_image = imread('vali-set/vali-set/'+test_labels[0][i])
        im = scipy.misc.imresize(content_image, (32,32))
        test_images_resize.append(im.astype('uint8'))


# Define new training & testing set (resized)        
x_train_resize = (np.asarray(train_images_resize)).reshape(len(train_images_resize), len(train_images_resize[0]), len(train_images_resize[0]), 1)
x_test_resize = (np.asarray(test_images_resize)).reshape(len(test_images_resize), len(test_images_resize[0]), len(test_images_resize[0]), 1)

/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [12]:
# Check shape of the new inputs
print('Train data: ',x_train_resize.shape)
print('Test data: ',x_test_resize.shape)

Train data:  (37882, 32, 32, 1)
Test data:  (6262, 32, 32, 1)


In [13]:
# Build a modified LeNet architecture with additional Dropout, batch normalization, Relu activiation function
# Details of the full model is decribed in the report, we used a different model name 'modellenet6' for this model
# When conduct experiment, we could simply delete the layer by adding '#' tounscript orline add a layer by using 'modellent6.add()'
# Results from the experiments were also included in the report

#### This cell showed the optimal model and hyper-parameter settings ####

modellenet6 = Sequential()

modellenet6.add(Conv2D(6, (5, 5), activation='relu', input_shape=(32, 32, 1)))
modellenet6.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
modellenet6.add(Dropout(0.25))
modellenet6.add(BatchNormalization())

modellenet6.add(Conv2D(16, (5, 5), activation='relu'))
modellenet6.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
modellenet6.add(Dropout(0.25))
modellenet6.add(BatchNormalization())

modellenet6.add(Flatten())
modellenet6.add(Dense(256, activation='relu'))
modellenet6.add(Dropout(0.25))
modellenet6.add(BatchNormalization())
modellenet6.add(Dense(62, activation='softmax'))


adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# Compiled the model
modellenet6.compile(loss='categorical_crossentropy', optimizer= adam, metrics=['accuracy'])

# Check runtime
start = time.time()

# Train model 
# Using the new resized datasets
modellenet6.fit(x_train_resize,y_train,batch_size=256,epochs=35,
          validation_data=(x_test_resize, y_test),
          verbose=1)
score = modellenet6.evaluate(x_test_resize, y_test, verbose=0)
print('Test loss: {:0.2f}%'.format(score[0]*100))
print('Test accuracy: {:0.2f}%'.format(score[1]*100))

# Print runtime
end = time.time()
print('Total runtime is {:0.2f} minutes'.format(int(end-start)/60))

Train on 37882 samples, validate on 6262 samples
Epoch 1/35
37882/37882 [==============================] - 4s 108us/step - loss: 2.1132 - acc: 0.4666 - val_loss: 0.9102 - val_acc: 0.7472
Epoch 2/35
37882/37882 [==============================] - 3s 79us/step - loss: 1.0684 - acc: 0.6978 - val_loss: 0.7093 - val_acc: 0.7866
Epoch 3/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.8629 - acc: 0.7454 - val_loss: 0.6231 - val_acc: 0.8109
Epoch 4/35
30976/37882 [=======================>......] - ETA: 0s - loss: 0.7742 - acc: 0.7638

37882/37882 [==============================] - 3s 78us/step - loss: 0.7665 - acc: 0.7654 - val_loss: 0.5697 - val_acc: 0.8175
Epoch 5/35
37882/37882 [==============================] - 3s 77us/step - loss: 0.7038 - acc: 0.7794 - val_loss: 0.5320 - val_acc: 0.8296
Epoch 6/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.6599 - acc: 0.7905 - val_loss: 0.5078 - val_acc: 0.8374
Epoch 7/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.6218 - acc: 0.7983 - val_loss: 0.4780 - val_acc: 0.8441
Epoch 8/35
37882/37882 [==============================] - 3s 79us/step - loss: 0.5905 - acc: 0.8059 - val_loss: 0.4647 - val_acc: 0.8457
Epoch 9/35
12544/37882 [========>.....................] - ETA: 1s - loss: 0.5599 - acc: 0.8124

37882/37882 [==============================] - 3s 78us/step - loss: 0.5654 - acc: 0.8119 - val_loss: 0.4465 - val_acc: 0.8473
Epoch 10/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.5430 - acc: 0.8169 - val_loss: 0.4352 - val_acc: 0.8491
Epoch 11/35
37882/37882 [==============================] - 3s 77us/step - loss: 0.5232 - acc: 0.8214 - val_loss: 0.4192 - val_acc: 0.8499
Epoch 12/35
37882/37882 [==============================] - 3s 77us/step - loss: 0.5028 - acc: 0.8258 - val_loss: 0.4017 - val_acc: 0.8601
Epoch 13/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.4880 - acc: 0.8280 - val_loss: 0.3908 - val_acc: 0.8644
Epoch 14/35
 7936/37882 [=====>........................] - ETA: 2s - loss: 0.4474 - acc: 0.8427

37882/37882 [==============================] - 3s 79us/step - loss: 0.4759 - acc: 0.8310 - val_loss: 0.3806 - val_acc: 0.8652
Epoch 15/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.4690 - acc: 0.8329 - val_loss: 0.3780 - val_acc: 0.8694
Epoch 16/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.4536 - acc: 0.8380 - val_loss: 0.3792 - val_acc: 0.8599
Epoch 17/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.4383 - acc: 0.8421 - val_loss: 0.3788 - val_acc: 0.8676
Epoch 18/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.4394 - acc: 0.8395 - val_loss: 0.3538 - val_acc: 0.8746
Epoch 19/35
 7936/37882 [=====>........................] - ETA: 2s - loss: 0.4329 - acc: 0.8396

37882/37882 [==============================] - 3s 77us/step - loss: 0.4274 - acc: 0.8438 - val_loss: 0.3507 - val_acc: 0.8718
Epoch 20/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.4183 - acc: 0.8458 - val_loss: 0.3503 - val_acc: 0.8698
Epoch 21/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.4140 - acc: 0.8467 - val_loss: 0.3412 - val_acc: 0.8777
Epoch 22/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.4035 - acc: 0.8500 - val_loss: 0.3394 - val_acc: 0.8753
Epoch 23/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.3998 - acc: 0.8499 - val_loss: 0.3377 - val_acc: 0.8729
Epoch 24/35
 7936/37882 [=====>........................] - ETA: 2s - loss: 0.3884 - acc: 0.8551

37882/37882 [==============================] - 3s 78us/step - loss: 0.3940 - acc: 0.8525 - val_loss: 0.3229 - val_acc: 0.8786
Epoch 25/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.3900 - acc: 0.8531 - val_loss: 0.3271 - val_acc: 0.8775
Epoch 26/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.3853 - acc: 0.8549 - val_loss: 0.3270 - val_acc: 0.8762
Epoch 27/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.3828 - acc: 0.8531 - val_loss: 0.3278 - val_acc: 0.8758
Epoch 28/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.3719 - acc: 0.8581 - val_loss: 0.3257 - val_acc: 0.8737
Epoch 29/35
 7936/37882 [=====>........................] - ETA: 2s - loss: 0.3691 - acc: 0.8598

37882/37882 [==============================] - 3s 78us/step - loss: 0.3694 - acc: 0.8601 - val_loss: 0.3143 - val_acc: 0.8845
Epoch 30/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.3647 - acc: 0.8597 - val_loss: 0.3153 - val_acc: 0.8802
Epoch 31/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.3615 - acc: 0.8606 - val_loss: 0.3158 - val_acc: 0.8818
Epoch 32/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.3536 - acc: 0.8629 - val_loss: 0.3133 - val_acc: 0.8801
Epoch 33/35
37882/37882 [==============================] - 3s 78us/step - loss: 0.3487 - acc: 0.8649 - val_loss: 0.3127 - val_acc: 0.8837
Epoch 34/35
 7936/37882 [=====>........................] - ETA: 2s - loss: 0.3489 - acc: 0.8666

37882/37882 [==============================] - 3s 77us/step - loss: 0.3546 - acc: 0.8620 - val_loss: 0.3072 - val_acc: 0.8841
Epoch 35/35
37882/37882 [==============================] - 3s 77us/step - loss: 0.3490 - acc: 0.8633 - val_loss: 0.2969 - val_acc: 0.8892
Test loss: 29.69%
Test accuracy: 88.92%
Total runtime is 1.77 minutes
